<a href="https://colab.research.google.com/github/Anushka-GA/AI-ML-Month/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
movies_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/movie.csv')

In [ ]:
movies_data.columns


Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [ ]:
movies_data['cast']=movies_data['actor_1_name']+', '+ movies_data['actor_2_name']+', '+ movies_data['actor_3_name']

In [ ]:
movies_data['cast']

0              CCH Pounder, Joel David Moore, Wes Studi
1            Johnny Depp, Orlando Bloom, Jack Davenport
2       Christoph Waltz, Rory Kinnear, Stephanie Sigman
3       Tom Hardy, Christian Bale, Joseph Gordon-Levitt
4                                                   NaN
                             ...                       
5038           Eric Mabius, Daphne Zuniga, Crystal Lowe
5039          Natalie Zea, Valorie Curry, Sam Underwood
5040         Eva Boehnke, Maxwell Moody, David Chandler
5041              Alan Ruck, Daniel Henney, Eliza Coupe
5042            John August, Brian Herzlinger, Jon Gunn
Name: cast, Length: 5043, dtype: object

Data Collection and Pre-Processing

In [ ]:
# printing the first 5 rows of the dataframe
movies_data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,cast
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,"CCH Pounder, Joel David Moore, Wes Studi"
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0,"Johnny Depp, Orlando Bloom, Jack Davenport"
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000,"Christoph Waltz, Rory Kinnear, Stephanie Sigman"
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000,"Tom Hardy, Christian Bale, Joseph Gordon-Levitt"
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0,NaN


In [ ]:

movies_data.isnull().sum()

color                         19
director_name                104
num_critic_for_reviews        50
duration                      15
director_facebook_likes      104
actor_3_facebook_likes        23
actor_2_name                  13
actor_1_facebook_likes         7
gross                        884
genres                         0
actor_1_name                   7
movie_title                    0
num_voted_users                0
cast_total_facebook_likes      0
actor_3_name                  23
facenumber_in_poster          13
plot_keywords                153
movie_imdb_link                0
num_user_for_reviews          21
language                      12
country                        5
content_rating               303
budget                       492
title_year                   108
actor_2_facebook_likes        13
imdb_score                     0
aspect_ratio                 329
movie_facebook_likes           0
cast                          23
dtype: int64

In [ ]:
# number of rows and columns in the data frame

movies_data.shape

(5043, 29)

In [ ]:
movies_data['index'] = np.arange(movies_data.shape[0])

In [ ]:
# selecting the relevant features for recommendation

selected_features = ['movie_title','plot_keywords','genres','cast','director_name', 'imdb_score']
print(selected_features)


['movie_title', 'plot_keywords', 'genres', 'cast', 'director_name', 'imdb_score']


In [ ]:
# replacing the null valuess with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [ ]:
# combining all the 5 selected features

combined_features = movies_data['genres']+' '+movies_data['plot_keywords']+' '+movies_data['movie_title']+' '+movies_data['cast']+' '+movies_data['director_name']

In [ ]:
print(combined_features)

0       Action|Adventure|Fantasy|Sci-Fi avatar|future|...
1       Action|Adventure|Fantasy goddess|marriage cere...
2       Action|Adventure|Thriller bomb|espionage|seque...
3       Action|Thriller deception|imprisonment|lawless...
4       Documentary  Star Wars: Episode VII - The Forc...
                              ...                        
5038    Comedy|Drama fraud|postal worker|prison|theft|...
5039    Crime|Drama|Mystery|Thriller cult|fbi|hideout|...
5040    Drama|Horror|Thriller  A Plague So Pleasant  E...
5041    Comedy|Drama|Romance  Shanghai Calling  Alan R...
5042    Documentary actress name in title|crush|date|f...
Length: 5043, dtype: object


In [ ]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

  (0, 2422)	0.20394950210334736
  (0, 7577)	0.13304022972354254
  (0, 14337)	0.25892617046234945
  (0, 15964)	0.21157184585595457
  (0, 10160)	0.1969178172973861
  (0, 3854)	0.12846364019581508
  (0, 7757)	0.1917658695733859
  (0, 11755)	0.2543648816849952
  (0, 2666)	0.2543648816849952
  (0, 11192)	0.3017192062913901
  (0, 10473)	0.25892617046234945
  (0, 9397)	0.23524900815915203
  (0, 5762)	0.18851017632429712
  (0, 1086)	0.5560840879763853
  (0, 5347)	0.10587443369480126
  (0, 13155)	0.10592975180346924
  (0, 5202)	0.10598515964146879
  (0, 309)	0.0920979590276485
  (0, 249)	0.08433557417521659
  (1, 15585)	0.23460774117342847
  (1, 6187)	0.22457462782671514
  (1, 3851)	0.2308969021045635
  (1, 7532)	0.16494803544769193
  (1, 1806)	0.21697664414955195
  (1, 10982)	0.22183327194834548
  :	:
  (5041, 4470)	0.06807342776657829
  (5041, 3784)	0.41979734963220433
  (5041, 423)	0.2208328958628896
  (5041, 3226)	0.08133844689983796
  (5041, 12670)	0.10275224288144628
  (5042, 6835)	0.2636

Cosine Similarity

In [ ]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.02474363 0.01733444 ... 0.01908966 0.         0.        ]
 [0.02474363 1.         0.01598803 ... 0.         0.         0.        ]
 [0.01733444 0.01598803 1.         ... 0.00774793 0.         0.        ]
 ...
 [0.01908966 0.         0.00774793 ... 1.         0.00447643 0.        ]
 [0.         0.         0.         ... 0.00447643 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [ ]:
print(similarity.shape)

(5043, 5043)


Getting the movie name from the user

In [ ]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : thriller


In [ ]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['movie_title'].tolist()
print(list_of_all_titles)

['Avatar\xa0', "Pirates of the Caribbean: At World's End\xa0", 'Spectre\xa0', 'The Dark Knight Rises\xa0', 'Star Wars: Episode VII - The Force Awakens\xa0            ', 'John Carter\xa0', 'Spider-Man 3\xa0', 'Tangled\xa0', 'Avengers: Age of Ultron\xa0', 'Harry Potter and the Half-Blood Prince\xa0', 'Batman v Superman: Dawn of Justice\xa0', 'Superman Returns\xa0', 'Quantum of Solace\xa0', "Pirates of the Caribbean: Dead Man's Chest\xa0", 'The Lone Ranger\xa0', 'Man of Steel\xa0', 'The Chronicles of Narnia: Prince Caspian\xa0', 'The Avengers\xa0', 'Pirates of the Caribbean: On Stranger Tides\xa0', 'Men in Black 3\xa0', 'The Hobbit: The Battle of the Five Armies\xa0', 'The Amazing Spider-Man\xa0', 'Robin Hood\xa0', 'The Hobbit: The Desolation of Smaug\xa0', 'The Golden Compass\xa0', 'King Kong\xa0', 'Titanic\xa0', 'Captain America: Civil War\xa0', 'Battleship\xa0', 'Jurassic World\xa0', 'Skyfall\xa0', 'Spider-Man 2\xa0', 'Iron Man 3\xa0', 'Alice in Wonderland\xa0', 'X-Men: The Last Stand\

In [ ]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Killers\xa0']


In [ ]:
close_match = find_close_match[0]
print(close_match)

Killers 


In [ ]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.movie_title == close_match]['index'].values[0]
print(index_of_the_movie)

547


In [ ]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.007619020107009557), (1, 0.0724122755306345), (2, 0.01564414703377103), (3, 0.042382524448606566), (4, 0.008011479143019671), (5, 0.008121269482856559), (6, 0.016419813179908035), (7, 0.03140113102338164), (8, 0.041743908161011714), (9, 0.005970787327790331), (10, 0.022558909786221777), (11, 0.007503436378222991), (12, 0.013654698391297003), (13, 0.012364061877035776), (14, 0.0343781865235801), (15, 0.024832116367850383), (16, 0.0102409814296806), (17, 0.03673220935341852), (18, 0.03399262884681022), (19, 0.013558244118300648), (20, 0.015970102415535956), (21, 0.012022063886317205), (22, 0.007169323125106464), (23, 0.011284061441652898), (24, 0.006784231235399177), (25, 0.014725963265896041), (26, 0.008572679489503927), (27, 0.042691851602125276), (28, 0.013403733056874936), (29, 0.012361334129902656), (30, 0.01357494918628743), (31, 0.01914950979892148), (32, 0.02936393680523734), (33, 0.0), (34, 0.01887908673188372), (35, 0.005008170129345874), (36, 0.01385316823796219), (37, 

In [ ]:
len(similarity_score)

5043

In [ ]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(547, 1.0), (189, 0.20630661899397956), (436, 0.17744484623186618), (2284, 0.16411740569183902), (1266, 0.16388769479666057), (550, 0.1596460337009633), (4913, 0.15829772095876524), (2219, 0.15616367737470457), (409, 0.15437778854279366), (5008, 0.14511665614018565), (1227, 0.13196314731415912), (2144, 0.12241541017217704), (442, 0.12237068483421584), (2698, 0.1214750014107309), (2352, 0.11773050480217144), (302, 0.11673175280115414), (1418, 0.11659664831739941), (2623, 0.11342676813675236), (1412, 0.11323977774048455), (457, 0.11259383856067212), (128, 0.11079465297162791), (1083, 0.11020756853881933), (1340, 0.1096320164349255), (2602, 0.10665650461525464), (3286, 0.10475370394954295), (2980, 0.10367871695852035), (714, 0.1025959982825561), (3925, 0.10221817290723466), (3384, 0.10163870434327277), (416, 0.10151078262441379), (4092, 0.09874517452049084), (1333, 0.09857028852315261), (3424, 0.09786708730796988), (2035, 0.09767268561506495), (4317, 0.09729640028390267), (1138, 0.097268

In [ ]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['movie_title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Killers 
2 . War of the Worlds 
3 . Bruce Almighty 
4 . Be Kind Rewind 
5 . Drillbit Taylor 
6 . Monster House 
7 . Good Dick 
8 . High Fidelity 
9 . Enemy of the State 
10 . Clerks 
11 . Shall We Dance 
12 . Away We Go 
13 . Over the Hedge 
14 . Home Fries 
15 . Survivor 
16 . Where the Wild Things Are 
17 . In & Out 
18 . Beetlejuice 
19 . 21 
20 . Road to Perdition 
21 . Mad Max: Fury Road 
22 . Monster-in-Law 
23 . The Ugly Truth 
24 . 3 Men and a Baby 
25 . Child's Play 
26 . For Your Consideration 
27 . Chicken Little 
28 . Waiting for Guffman 
29 . The Good Girl 


Movie Recommendation Sytem

In [ ]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['movie_title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.movie_title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['movie_title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : bat man
Movies suggested for you : 

1 . Batman 
2 . Batman Returns 
3 . Superman IV: The Quest for Peace 
4 . Batman & Robin 
5 . Constantine             
6 . The Ladies Man 
7 . Young Guns 
8 . Batman: The Movie 
9 . Tango & Cash 
10 . Dear Wendy 
11 . The Dark Knight 
12 . Private Benjamin 
13 . Sphinx 
14 . Flash Gordon 
15 . Out of Africa 
16 . Batman Forever 
17 . Steamboy 
18 . 8 Heads in a Duffel Bag 
19 . The Omega Code 
20 . Raiders of the Lost Ark 
21 . Batman Begins 
22 . Police Academy 
23 . Spawn 
24 . The Marine 4: Moving Target 
25 . Swelter 
26 . Harry Brown 
27 . Fantastic Four 
28 . Fantastic Four 
29 . City by the Sea 
